<a href="https://colab.research.google.com/github/rakshitha-ram/course-dbt/blob/main/Automation_for_updating_price_changes_into_dwh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Center Pricing Output **

---
This code generates the pricing file for data ceneter based pricing


In [24]:
import pandas as pd
import hashlib

# Load the two input CSV files
tiers_df = pd.read_csv('dc_tier_price_info.csv')  # The file with tiers and prices
product_info_df = pd.read_csv('product_info.csv')  # The file with product details

# Define the output file name
output_file = 'output_dc.csv'

# column names for product_info_df
product_info_df.columns = ['product_platform_id', 'vendor', 'name']

# Prepare the list of tiers from the first CSV file
# The first column in tiers_df contains the tier values (e.g., 10, 50, 100...)
tiers = tiers_df.iloc[:, 0].tolist()  # First column as list of tiers

# Generate product_id using the formula: CONCAT('pro_', TO_HEX(SHA256(CONCAT(product_platform_id, '_', 'atlassian', '_', 'vendor'))))
def generate_product_id(product_platform_id, vendor):
    sha_string = f'{product_platform_id}_atlassian_{vendor}'
    product_id_hash = hashlib.sha256(sha_string.encode()).hexdigest()
    return f"pro_{product_id_hash}"

# Prepare the list of columns for the output CSV
csv_columns = [
    "record_id", "product_id", "product_platform_id", "tier_string", "tier",
    "tier_bucket", "tier_bucket_lower_limit", "tier_bucket_upper_limit",
    "hosting", "tempo_price", "tempo_bucket_tier_price", "changed_in_month",
    "year", "date", "sale_type", "name", "is_current_price", "platform", "vendor"
]

# Prepare the output data
output_data = []

# Process each product and tier
for _, product_row in product_info_df.iterrows():
    product_platform_id = product_row['product_platform_id']
    vendor = product_row['vendor']
    name = product_row['name']

    # Find the corresponding column in tiers_df for this product's pricing
    if product_platform_id in tiers_df.columns:
        product_prices = tiers_df[product_platform_id]  # Get the price column for this product

        for tier, price in zip(tiers, product_prices):
            if pd.isna(price):  # Handle missing prices by setting it to "no price available"
                price = "no price available"
            elif price == 0:  # Skip rows where the price is 0
                continue

            product_id = generate_product_id(product_platform_id, vendor)

            # Create a row for "New" sale type
            new_row = {
                "record_id": "",
                "product_id": product_id,
                "product_platform_id": product_platform_id,
                "tier_string": f"{tier} users",
                "tier": tier,
                "tier_bucket": "",
                "tier_bucket_lower_limit": "",
                "tier_bucket_upper_limit": "",
                "hosting": "Data Center",
                "tempo_price": price,
                "tempo_bucket_tier_price": "",
                "changed_in_month": 10,  # Month for "New" sale
                "year": 2024,
                "date": "2024-10-16",  # Date for "New" sale
                "sale_type": "New",
                "name": name,
                "is_current_price": "",
                "platform": "atlassian",
                "vendor": vendor
            }

            # Create a row for "Renewal" sale type
            renewal_row = {
                "record_id": "",
                "product_id": product_id,
                "product_platform_id": product_platform_id,
                "tier_string": f"{tier} users",
                "tier": tier,
                "tier_bucket": "",
                "tier_bucket_lower_limit": "",
                "tier_bucket_upper_limit": "",
                "hosting": "Data Center",
                "tempo_price": price,
                "tempo_bucket_tier_price": "",
                "changed_in_month": 12,  # Month for "Renewal" sale
                "year": 2024,
                "date": "2024-12-16",  # Date for "Renewal" sale
                "sale_type": "Renewal",
                "name": name,
                "is_current_price": "",
                "platform": "atlassian",
                "vendor": vendor
            }

            # Append the new and renewal rows to the output data
            output_data.append(new_row)
            output_data.append(renewal_row)

# Create a DataFrame for the output
output_df = pd.DataFrame(output_data, columns=csv_columns)

# Write the DataFrame to a CSV file
output_df.to_csv(output_file, index=False)

print(f"CSV file '{output_file}' generated successfully!")


CSV file 'output_dc.csv' generated successfully!


**Cloud Pricing Output **

---
This code generates the pricing file for cloud based pricing


In [23]:
import pandas as pd
import hashlib

# Load the two input CSV files
tiers_df = pd.read_csv('cloud_tier_price_info.csv')  # The file with tiers, prices, and lower/upper tier limits
product_info_df = pd.read_csv('product_info.csv')  # The file with product details

# Define the output file name
output_file = 'output_cloud.csv'

# column names for product_info_df
product_info_df.columns = ['product_platform_id', 'vendor', 'name']

# Prepare the list of upper and lower tiers from the first CSV file
# 'lower tier' and 'upper tier' columns exist in tiers_df
lower_tiers = tiers_df['lower tier']
upper_tiers = tiers_df['upper tier']
product_ids = tiers_df.columns[2:]  # Assuming the first two columns are lower and upper tiers

# Generate product_id using the formula: CONCAT('pro_', TO_HEX(SHA256(CONCAT(product_platform_id, '_', 'atlassian', '_', 'vendor'))))
def generate_product_id(product_platform_id, vendor):
    sha_string = f'{product_platform_id}_atlassian_{vendor}'
    product_id_hash = hashlib.sha256(sha_string.encode()).hexdigest()
    return f"pro_{product_id_hash}"

# Prepare the list of columns for the output CSV
csv_columns = [
    "record_id", "product_id", "product_platform_id", "tier_string", "tier",
    "tier_bucket", "tier_bucket_lower_limit", "tier_bucket_upper_limit",
    "hosting", "tempo_price", "tempo_bucket_tier_price", "changed_in_month",
    "year", "date", "sale_type", "name", "is_current_price", "platform", "vendor"
]

# Prepare the output data
output_data = []

# Process each product and its corresponding lower/upper tier limits
for _, product_row in product_info_df.iterrows():
    product_platform_id = product_row['product_platform_id']
    vendor = product_row['vendor']
    name = product_row['name']

    # Check if product_platform_id exists in tiers_df
    if product_platform_id in product_ids:
        product_prices = tiers_df[product_platform_id]  # Get the price column for this product

        for lower_limit, upper_limit, price in zip(lower_tiers, upper_tiers, product_prices):
            if pd.isna(price):  # Handle missing prices
                price = ""  # Set price to blank
            elif price == 0:  # Skip rows where the price is 0
                continue

            product_id = generate_product_id(product_platform_id, vendor)

            # Create a row for "New" sale type
            new_row = {
                "record_id": "",
                "product_id": product_id,
                "product_platform_id": product_platform_id,
                "tier_string": "",  # Blank
                "tier": "",  # Blank
                "tier_bucket": f"{lower_limit}-{upper_limit} Users",  # Format as "lower-upper Users"
                "tier_bucket_lower_limit": lower_limit,  # Comes from lower tier column
                "tier_bucket_upper_limit": upper_limit,  # Comes from upper tier column
                "hosting": "Cloud",  # Set hosting to Cloud
                "tempo_price": "",  # Blank
                "tempo_bucket_tier_price": price,  # Price from tiers_df
                "changed_in_month": 10,  # Month for "New" sale
                "year": 2024,
                "date": "2024-10-16",  # Date for "New" sale
                "sale_type": "New",
                "name": name,
                "is_current_price": "",  # Blank
                "platform": "atlassian",
                "vendor": vendor
            }

            # Create a row for "Renewal" sale type
            renewal_row = {
                "record_id": "",
                "product_id": product_id,
                "product_platform_id": product_platform_id,
                "tier_string": "",  # Blank
                "tier": "",  # Blank
                "tier_bucket": f"{lower_limit}-{upper_limit} Users",  # Format as "lower-upper Users"
                "tier_bucket_lower_limit": lower_limit,  # Comes from lower tier column
                "tier_bucket_upper_limit": upper_limit,  # Comes from upper tier column
                "hosting": "Cloud",  # Set hosting to Cloud
                "tempo_price": "",  # Blank
                "tempo_bucket_tier_price": price,  # Price from tiers_df
                "changed_in_month": 12,  # Month for "Renewal" sale
                "year": 2024,
                "date": "2024-12-16",  # Date for "Renewal" sale
                "sale_type": "Renewal",
                "name": name,
                "is_current_price": "",  # Blank
                "platform": "atlassian",
                "vendor": vendor
            }

            # Append the new and renewal rows to the output data
            output_data.append(new_row)
            output_data.append(renewal_row)

# Create a DataFrame for the output
output_df = pd.DataFrame(output_data, columns=csv_columns)

# Write the DataFrame to a CSV file
output_df.to_csv(output_file, index=False)

print(f"CSV file '{output_file}' generated successfully!")


CSV file 'output_cloud.csv' generated successfully!
